In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
from collections import namedtuple
from spiketimes.df.binning import which_bin
from spiketimes.df.alignment import align_around

In [2]:
report_dir = r"C:\Users\Rory\Google Drive\ah_vids\PLPAG_ChR\freeze_analysis\conditioning_reports"
output_dir = report_dir
metadata_file = r"C:\Users\Rory\Google Drive\ah_vids\PLPAG_ChR\metadata\align.csv"
groups_file = r"C:\Users\Rory\Google Drive\ah_vids\PLPAG_ChR\metadata\animals.csv"

In [3]:
report_dir = Path(report_dir)
output_dir = Path(output_dir)
df_groups = pd.read_csv(groups_file)
df_meta = pd.read_csv(metadata_file)
report_paths = [*report_dir.glob("*.csv")]
df_reports = (
    pd.concat([*map(pd.read_csv, report_paths)])
    .assign(
        animal_name=lambda x: x.video_name.apply(
            lambda y: y.split(".")[0]
        )
    )
)

In [4]:
def align_to_experiment(df, df_meta):
    animal_name = df.animal_name.unique()[0]
    dfm = df_meta.loc[lambda x: (x.animal_name == animal_name) &
                      (x.experimental_timepoint_name == "first_cs")
                     ]
    exp_time, vid_time = dfm.timepoint_in_experiment.values[0], dfm.time_in_video.values[0]
    offset = exp_time - vid_time
    return df.assign(exp_time = lambda x: x.time.add(offset))

In [5]:
df_reports_aligned = (
    df_reports
    .groupby("animal_name")
    .apply(
        lambda x: align_to_experiment(x, df_meta=df_meta)
    )
    .reset_index(drop=True)
)

In [6]:
def _get_block(baseline_numsec, cs_numsec, iti_numsec, postexp_numsec, num_trials):
    Block = namedtuple("Block", ["name", "length"])
    baseline = Block(name="Baseline", length=baseline_numsec)
    cs = Block(name="CS", length=cs_numsec)
    iti = Block(name="ITI", length=iti_numsec)
    postexp = Block(name="PostExperiment", length=postexp_numsec)
    blocks = [baseline]
    blocks.extend([cs, iti] * num_trials)
    blocks.append(postexp)
    block_lengths = np.array([*map(lambda x: x.length, blocks)])
    block_timepoints = np.insert(0, [1], np.cumsum(block_lengths))
    return blocks, block_timepoints

def _get_blocktype(df, blocks):
    return df.bin_idx.apply(
        lambda x: np.nan if np.isnan(x) else blocks[int(x)].name
    )

def _get_trialnum(idx):
    if np.isnan(idx):
        return np.nan
    if not (idx > 0 and idx < 61):
        return np.nan
    return int(np.ceil(idx // 2))

def align_to_blocks(
        df, 
        baseline_numsec=180, 
        cs_numsec=30, 
        iti_numsec=10, 
        postexp_numsec=180, 
        num_trials=30
):
    blocks, blocktimepoints = _get_block(
        baseline_numsec=baseline_numsec, 
        cs_numsec=cs_numsec, 
        iti_numsec=iti_numsec, 
        postexp_numsec=postexp_numsec, 
        num_trials=30
    )
    max_latency = np.max([baseline_numsec, cs_numsec, iti_numsec, postexp_numsec])
    df = (
        which_bin(
            df, spiketimes_col="exp_time", bin_edges=blocktimepoints, max_latency=max_latency
        )
        .assign(
            block_type = lambda x: _get_blocktype(x, blocks=blocks),
            trial_num =  lambda x: x.bin_idx.apply(_get_trialnum)
        )
        .drop(["bin_idx", "bin_values"], axis=1)
    )
    df = align_around(df, data_colname="exp_time", events=blocktimepoints)
    return df

In [7]:
df_out = align_to_blocks(
    df_reports_aligned, 
    baseline_numsec=180, 
        cs_numsec=30, 
        iti_numsec=10, 
        postexp_numsec=180, 
        num_trials=30
)

In [9]:
df_out.merge(df_groups).to_csv(output_dir / "PLPAG_ChR_tidy.csv", index=False)